In [1]:
print("*********| Hydrogen Bonding Analysis Script | **********")
print("A python script to analysis hydrogen bonding")
print("Require python >=3, last-tested with Python=3.8.3")
print("Written by Ajay Khanna")
print("Nov.26.2021 | UC-Merced | Dr. Isborb's Lab")
print()
#----------------Importing Libraries------------------------
import pandas as pd
import numpy as np
import os.path
rootname = os.getcwd()
#----------------Setting up inital variable-----------------
filename = rootname + str(input('Enter the name of file: '))
nDyeAtoms = int(input('Enter total number of atoms in dye: '))
nQAtoms = int(input('Enter total number of atoms: '))
data = pd.read_csv(filename, skiprows=1,
                   skipinitialspace=True,
                   nrows=nQAtoms,
                   delimiter='\t',
                   names=('ATOMS', 'X', 'Y', 'Z'))
data[0:nDyeAtoms] # Display Dye ATOMS
#---------------------------------------------------

*********| Hydrogen Bonding Analysis Script | **********
A python script to analysis hydrogen bonding
Require python >=3, last-tested with Python=3.8.3
Written by Ajay Khanna
Nov.26.2021 | UC-Merced | Dr. Isborb's Lab



/home/akhanna2/data/conda_envs/openmm_env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/akhanna2/data/conda_envs/openmm_env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Enter the name of file:  nb_dmso_5ang_qm_ex_opt_vde.xyz
Enter total number of atoms in dye:  17
Enter total number of atoms:  157


,ATOMS,X,Y,Z
0,C,1.143808,5.093382,23.122179
1,C,1.828220,5.195758,24.339434
2,C,3.199144,5.044226,24.398635
3,C,1.861583,4.785289,21.904984
4,C,3.254770,4.579146,21.986105
5,C,3.971804,4.734650,23.215209
6,N,1.426910,4.588641,20.680913
7,N,3.738085,4.215471,20.797739
8,O,2.600277,4.214801,19.973614
9,N,-0.274137,5.249471,23.108070


In [2]:
# Function to Find Distance b/w
# Acceptor Hydrogen Atom and Solvent Donor Atom
def find_distance(hatom_array, d_atom_array):
    # hatom_array : Acceptor H-Atom XYZ Coordinates
    # d_atom_array: Donor Atom [O, N, F, Cl] XYZ Coordinates
    # distance: Distance b/w D----H-A (in Angstrom)    
    distance = np.linalg.norm(hatom_array-d_atom_array)
    return distance

# Function to Find angle b/w
# Acceptor, Hydrogen Atom and Solvent Donor Atom
def find_angle(a_atom_array, hatom_array, d_atom_array):
    # a_atom_array: Acceptor Atom Bonded To H XYZ Coodinates
    # angle: Inverse Cosine Angle Between D--H-A (Degrees)
    # distance = SQRT(((x2-x1)^2+(y2-y1)^2+(z2-z1)^2))
    # ba = d_atom_array - hatom_array
    # bc = d_atom_array - a_atom_array
    # angle = invcos(ba.bc / distance(ba) * distance(bc) )
    ha = a_atom_array - hatom_array
    hd = d_atom_array - hatom_array
    cosine_angle = np.dot(ha, hd) / (np.linalg.norm(ha) * np.linalg.norm(hd))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

In [3]:
coord = np.array(data.loc[:,'X':'Z']) # Full QM-System
solute = coord[0:nDyeAtoms] # indexing Dye
solvent = coord[nDyeAtoms:] # indexing Solvent

In [4]:
first_HBond = [] # 1st H-Bond length b/w Acceptor(A)-Dye and Donor(D)-Solvent
second_HBond = [] # 1st Bond Angle b/w A-H-D
first_HAngle = [] # 2nd H-Bond length b/w Acceptor(A)-Dye and Donor(D)-Solvent
second_HAngle = [] # 2nd Bond Angle b/w A-H-D
o_atom_coodinates = [] # Solvent Donor: O Atom XYZ Coordinates
o_atom_vmd_index = [] #Solvent Donor: O Atom index

for i in range(1, int(solvent.size/3)+1, 10): # A loog over Only Oxygen Atoms of the Solvent
    # Finding H---Bond Distance
    x = find_distance(solute[15], solvent[i])
    y = find_distance(solute[16], solvent[i])
    
    # Finding H---Bond Angle
    # This is an angle between [Acceptor, Hydrogen, Donor]
    x_angle = find_angle(solute[12], solute[15], solvent[i]) 
    y_angle = find_angle(solute[12], solute[16], solvent[i])
    first_HBond.append(x)
    second_HBond.append(y)
    first_HAngle.append(x_angle)
    second_HAngle.append(y_angle)
    o_atom_coodinates.append(solvent[i])
    o_atom_vmd_index.append(17 + i)

# Hydrogen Bond Length    
first_HBond = np.array(first_HBond)
second_HBond = np.array(second_HBond)
# Hydrogen Bond Angle 
# VMD Hydrogen Bond Angle Cut-off: 180 - angle <=20 then only it forms H-Bond irrespective of bond length
first_HAngle = 180. - np.array(first_HAngle) 
second_HAngle = 180. -  np.array(second_HAngle)
o_atom_coodinates = np.array(o_atom_coodinates)
# Isolating Solvent Oxygen Atoms XYZs and index
# Usfull in locating index for visualization in VMD
o_atom_x_coods = o_atom_coodinates[:,0]
o_atom_y_coods = o_atom_coodinates[:,1]
o_atom_z_coods = o_atom_coodinates[:,2]
o_atom_vmd_index = np.array(o_atom_vmd_index) # Array of O-Atom Index

In [5]:
# Creating A DataFrame: Off All Variable Arrays
h_bond_information = pd.DataFrame([o_atom_vmd_index, o_atom_x_coods, o_atom_y_coods, o_atom_z_coods, first_HBond, first_HAngle, second_HBond, second_HAngle],
                                  index=('VMD Index','X', 'Y', 'Z', 'First H---O(Å)', 'First O-H-N(θ)', 'Second H---O(Å)','Second O-H-N(θ)')).T
# Saving H_Bond Analysis
# Custom Format for Each Columns
h_bond_information['VMD Index'] = h_bond_information['VMD Index'].map(lambda x: '%2.0d' % x)
h_bond_information.to_csv('full_O-H_analysis.csv',sep='\t', float_format='%0.4f')

# Functions To Apply Conditional Formating To Highlight
# Favorable HBond Lengths &
# Favorabel HBond Angles
def hbond_angle_cutoff(column):    

    highlight = 'background-color: palegreen;'
    default = ''

    #maximum_in_column = column.min()

    # must return one string per cell in this column
    return [highlight if v < 20. else default for v in column]

def hbond_length_cutoff(column):    

    highlight = 'background-color: palegreen;'
    default = ''

    minimum_in_column = column.min()

    # must return one string per cell in this column
    return [highlight if v == minimum_in_column else default for v in column]

#h_bond_information.style.apply(maximum_value_in_column, subset=['First H---O(Å)', 'First O-H-N(θ)', 'Second H---O(Å)','Second O-H-N(θ)'], axis=0)
(h_bond_information.style
 .apply(hbond_length_cutoff, subset=['First H---O(Å)','Second H---O(Å)'], axis=0)
 .apply(hbond_angle_cutoff, subset=['First O-H-N(θ)','Second O-H-N(θ)'], axis=0))

,VMD Index,X,Y,Z,First H---O(Å),First O-H-N(θ),Second H---O(Å),Second O-H-N(θ)
0,18,3.88488,10.3082,24.2094,5.80017,86.6513,6.54812,129.149
1,28,6.83988,3.65248,21.1596,3.41076,132.765,1.78464,5.77064
2,38,5.00381,8.73825,20.0892,5.71364,121.409,5.10857,87.8467
3,48,4.40589,3.16727,26.2722,2.99718,75.2794,4.20563,145.188
4,58,2.50183,5.9369,15.1394,9.67704,158.457,8.22254,64.0325
5,68,1.12724,1.94648,15.5788,10.1883,176.266,8.67369,65.0375
6,78,-4.98494,6.55591,21.6542,11.1827,128.098,11.0756,121.306
7,88,2.96246,5.91183,27.5181,4.49623,67.2475,6.0165,175.818
8,98,6.81484,5.7309,25.4999,1.91286,17.6797,3.48603,131.02
9,108,-2.25185,2.44188,28.1756,9.2733,95.9409,10.058,145.715


In [12]:
extracting_hbonds_only = h_bond_information.loc[((h_bond_information['First O-H-N(θ)'] < 20.) | (h_bond_information['Second O-H-N(θ)'] < 20.))]
extracting_hbonds_only.to_csv('only_h_bond_analysis.csv',sep='\t', float_format='%0.4f')
(extracting_hbonds_only.style
 .apply(hbond_length_cutoff, subset=['First H---O(Å)','Second H---O(Å)'], axis=0)
 .apply(hbond_angle_cutoff, subset=['First O-H-N(θ)','Second O-H-N(θ)'], axis=0))

,VMD Index,X,Y,Z,First H---O(Å),First O-H-N(θ),Second H---O(Å),Second O-H-N(θ)
1,28,6.83988,3.65248,21.1596,3.41076,132.765,1.78464,5.77064
8,98,6.81484,5.7309,25.4999,1.91286,17.6797,3.48603,131.02
